In [15]:
# Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.models import resnet50
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
import numpy as np
import re
from io import StringIO
import os
import csv
from sklearn.metrics import f1_score

# Constants
IMAGE_PATH = './data/'
TRAINING_DATA_PATH = './train.csv'
TEST_DATA_PATH = './test.csv'
LABEL_COUNT = 19
BATCH_SIZE = 32
IMAGE_SHAPE = 224
THRESHOLD = 0.5
LR = 0.001
EPOCHS = 1
WEIGHT_DECAY = 0.001

# CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Using', device)

# Reading CSV Files
def read_csv(file_path):
    with open(file_path) as file:
        lines = [re.sub(r'([^,])"(\s*[^\n])', r'\1/"\2', line) for line in file]
        df = pd.read_csv(StringIO(''.join(lines)), escapechar="/")
    return df

df_train = read_csv(TRAINING_DATA_PATH)
df_test = read_csv(TEST_DATA_PATH)

# Dataset Class
class ImageDataset(Dataset):
    def __init__(self, df, image_path, transform=None, test=False):
        self.df = df
        self.image_path = image_path
        self.transform = transform
        self.test = test

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_path, self.df.iloc[idx, 0])
        image = Image.open(img_name).convert('RGB')
        if self.transform:
            image = self.transform(image)

        if self.test:
            return image
            
        label = torch.zeros(LABEL_COUNT, dtype=torch.float32)
        label_indices = torch.tensor([int(i) for i in self.df.iloc[idx, 1].split()])
        label[label_indices-1] = 1
        return image, label

# Transforms
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(IMAGE_SHAPE),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transform = transforms.Compose([
    transforms.Resize((IMAGE_SHAPE, IMAGE_SHAPE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Datasets and DataLoaders
train_dataset = ImageDataset(df_train, IMAGE_PATH, transform=train_transform)
test_dataset = ImageDataset(df_test, IMAGE_PATH, transform=test_transform, test=True)

# Splitting Dataset
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# ResNet Model
class ResNetModel(nn.Module):
    def __init__(self, num_classes):
        super(ResNetModel, self).__init__()
        self.resnet = resnet50(pretrained=True)
        for param in self.resnet.parameters():
            param.requires_grad = False

        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_features, num_classes)

        for param in self.resnet.fc.parameters():
            param.requires_grad = True
            
    def forward(self, x):
        return self.resnet(x)

# Model Initialization
model = ResNetModel(num_classes=LABEL_COUNT).to(device)

# Define optimizer and criterion
optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
criterion = nn.BCEWithLogitsLoss()
step_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

# Training Loop
def train(model, train_loader, optimizer, criterion):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

def validate(model, val_loader, criterion):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            preds = torch.sigmoid(outputs).cpu().detach().numpy()
            all_preds.extend(preds)
            all_targets.extend(labels.cpu().numpy())
    all_preds = np.array(all_preds)
    all_targets = np.array(all_targets)
    f1 = f1_score(all_targets, all_preds > THRESHOLD, average='micro')
    return running_loss / len(val_loader.dataset), f1

for epoch in range(EPOCHS):
    train(model, train_loader, optimizer, criterion)
    step_lr_scheduler.step()

val_loss, val_f1 = validate(model, val_loader, criterion)
print(f"Validation Loss: {val_loss:.4f}, F1 Score: {val_f1:.4f}")

# Testing Function
def predict(model, test_loader):
    model.eval()
    predictions = []
    with torch.no_grad():
        for inputs in test_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            preds = torch.sigmoid(outputs).cpu().detach().numpy()
            binary_preds = (preds > THRESHOLD).astype(int)
            labels = [' '.join([str(i+1) for i, val in enumerate(pred) if val == 1]) for pred in binary_preds]
            predictions.extend(labels)
    return predictions

# Output Predictions
predictions = predict(model, test_loader)

# Export Predictions
output_file = 'predictions.csv'
with open(output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['ImageID', 'Labels'])
    for idx, img_id in enumerate(df_test['ImageID']):
        writer.writerow([img_id, predictions[idx]])

print(f'Predictions exported to {output_file}.')


Using cpu


/Users/hewenkai/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hewenkai/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


KeyboardInterrupt: 